# Data extraction and aggregation

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

path = r'./../data/UNSW-NB15/'

features = pd.read_csv(path + "NUSW-NB15_features.csv", encoding='cp1252')

data = pd.concat([
    pd.read_csv(path + "UNSW-NB15_1.csv", low_memory=False, names=features.Name),
    pd.read_csv(path + "UNSW-NB15_2.csv", low_memory=False, names=features.Name),
    pd.read_csv(path + "UNSW-NB15_3.csv", low_memory=False, names=features.Name),
    pd.read_csv(path + "UNSW-NB15_4.csv", low_memory=False, names=features.Name)
])

data.info()

# Features Conversion

## Features which are unusable in the real world are dropped

the features are : `srcip`, `sport`, `dstip`

ip and port can vary and they can be faked using vpns

In [ ]:
data.drop(
	columns=["srcip", "sport", "dstip"], 
	inplace=True
)
data.info()

## Parsing `ct_ftp_cmd` to int

setting `-1` if the value was blank

In [ ]:
data["ct_ftp_cmd"] = data["ct_ftp_cmd"].apply(lambda x: int(x) if x != " " else -1)
data.info()

## Converting Hex to int

In [ ]:
data["dsport"] = data["dsport"].apply(lambda x: int(x, 16) if x != "-" else -1)
data.info()

## Setting `Nan` to `-1`

In [ ]:
data["ct_flw_http_mthd"].fillna(-1,  inplace=True)
data["is_ftp_login"].fillna(-1,  inplace=True)


## Converting the nominal features

todo: create the enum for each features

In [ ]:
nominal_features = ["proto", "state", "service"]

for nominal_feature in nominal_features:
		new_values, index = pd.factorize(data[nominal_feature])
		data[nominal_feature] = new_values

data.info()

# Data reduction

## Binary classication distribution

In [ ]:
data_summary_attack = pd.DataFrame(data.Label.value_counts())
data_summary_attack.columns.values[0] = "Occurrencies"

data_summary_attack["Percentage"] = data.Label.value_counts() / len(data)

data_summary_attack

We have a very unbalanced dataset. Now let's look at the attack classes

## Multiclass classificatin distribution

In [ ]:
data_summary_classes = pd.DataFrame(data.attack_cat.value_counts())
data_summary_classes.columns.values[0] = "Occurrencies"

attack_len = sum(data.Label)
data_summary_classes["Percentage"] = data.attack_cat.value_counts() / attack_len

data_summary_classes

also the classes of attack are very unbalanced

## Reduce the number of classes of attack

Let's first see the clusterability of the data set

In [ ]:
import pyclustertend as pyct

#Calculating the clusterability of the data frame in multiple samples
data2 = data[data["attack_cat"] != 'nan'].dropna().drop_duplicates()
avg_list = []
for sampling_size in range(10,51,10):
    sample_list = []
    for i in range(1,10,1):
        data_blob = data2.drop(columns=["Label", "attack_cat"]).sample(20_000)
        sample_list.append(pyct.hopkins(data_blob,sampling_size))
    print(f'Sampled {sampling_size}: {sum(sample_list) / len(sample_list)}')

The value is close to 0, so it'very clusterable

Data is clustered using k-means

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

to_plot = data[data["attack_cat"] != 'nan'].dropna().drop_duplicates()

to_plot = to_plot.drop(columns=["Label", "attack_cat"]).sample(50_000)

#to_plot = StandardScaler().fit_transform(to_plot)

silhouette_list = []
inertia_list=[]

for n_clusters in range(2,11):
    kmeans = KMeans(n_clusters=n_clusters)
    y_pred = kmeans.fit_predict(to_plot)

    # evaluate silhouette score
    silhouetteavg = silhouette_score(to_plot,y_pred)
    silhouette_list.append(silhouetteavg)

    # evaluate inertia
    inertia_list.append(kmeans.inertia_)


plt.tight_layout()

# plot silhouette and inertia trends w.r.t the number of clusters
fig, ax1 = plt.subplots()
ax1.set_xlabel('k')
ax1.set_ylabel('avg-silhouette', color='black')
ax1.plot(range(2,11),silhouette_list,'--ok')
ax1.tick_params(axis='y', labelcolor='black')
ax1.grid(axis='y')

ax2 = ax1.twinx()
ax2.set_ylabel('loss', color='red')
ax2.plot(range(2,11), inertia_list,'--or',alpha = 0.2)
ax2.tick_params(axis='y', labelcolor='red')

plt.tight_layout()  # otherwise the right y-label is slightly clipped

Using the silhouette method we have that the optimal number of clusters is 4-ish and 6-ish for only the attack categories, let's print the info of each cluster

In [ ]:
# data_cluster = data[data["attack_cat"] != 'nan'].drop(columns="Label").dropna().sample(50_000)
data_cluster = data[data["attack_cat"] != 'nan'].drop(columns="Label").dropna().drop_duplicates()
data_cluster.attack_cat = data_cluster.attack_cat.apply(lambda x: x.strip())
data_cluster.attack_cat = data_cluster.attack_cat.apply(lambda x: "Backdoor" if x == "Backdoors" else x)

# Clustering with 4 clusters
cluster1 = KMeans(n_clusters = 4)
data_cluster["cluster"] = cluster1.fit_predict(data_cluster.drop(columns="attack_cat"))

# conta il numero di elementi in ogni cluster
counts = data_cluster.groupby('cluster')['attack_cat'].value_counts()

#counts_df = pd.DataFrame(counts)
table = pd.DataFrame()
for i in range(0,4):
    table["C " + str(i)] = pd.DataFrame(counts[i])

# print(table)
# print(table.div(table.sum(axis=0), axis=1).round(4) * 100)
print(table.div(table.sum(axis=1), axis=0).round(4) * 100)

# Clustering with 7 clusters
cluster2 = KMeans(n_clusters = 7)
data_cluster["cluster"] = cluster2.fit_predict(data_cluster.drop(columns="attack_cat"))

# conta il numero di elementi in ogni cluster
counts = data_cluster.groupby('cluster')['attack_cat'].value_counts()

#counts_df = pd.DataFrame(counts)
table = pd.DataFrame()
for i in range(0,7):
    table["C " + str(i)] = pd.DataFrame(counts[i])

# print(table)
# print(table.div(table.sum(axis=0), axis=1).round(4) * 100)
print(table.div(table.sum(axis=1), axis=0).round(4) * 100)

# Clustering with 9 clusters
cluster3 = KMeans(n_clusters = 9)
data_cluster["cluster"] = cluster3.fit_predict(data_cluster.drop(columns="attack_cat"))

# conta il numero di elementi in ogni cluster
counts = data_cluster.groupby('cluster')['attack_cat'].value_counts()

#counts_df = pd.DataFrame(counts)
table = pd.DataFrame()
for i in range(0,9):
    table["C " + str(i)] = pd.DataFrame(counts[i])

# print(table)
# print(table.div(table.sum(axis=0), axis=1).round(4) * 100)
print(table.div(table.sum(axis=1), axis=0).round(4) * 100)

# Clustering with 15 clusters
cluster4 = KMeans(n_clusters = 15)
data_cluster["cluster"] = cluster4.fit_predict(data_cluster.drop(columns="attack_cat"))

# conta il numero di elementi in ogni cluster
counts = data_cluster.groupby('cluster')['attack_cat'].value_counts()

#counts_df = pd.DataFrame(counts)
table = pd.DataFrame()
for i in range(0,15):
    table["C " + str(i)] = pd.DataFrame(counts[i])

# print(table)
# print(table.div(table.sum(axis=0), axis=1).round(4) * 100)
print(table.div(table.sum(axis=1), axis=0).round(4) * 100)


[Shellcode] and [Reconnaissance] act similarly, the same goes for [Fuzzers] and [DoS], then [Analysis] and [Backdoor]. Then the [Worms] class can be eliminated because of the too few examples

we can see that the classes of attack are reduced to [Shellcode/Reconnaissance], [Fuzzers/DoS], [Analysis/Backdoor], [Generic] and [Exploits]

In [ ]:
data_reduced = data[data['attack_cat'] != 'nan'].dropna().drop_duplicates()
data_reduced = data_reduced[data_reduced['attack_cat'] != 'Worms']
# print(data_reduced)

data_reduced.attack_cat = data_reduced.attack_cat.apply(lambda x: str(x).strip())
data_reduced.attack_cat = data_reduced.attack_cat.apply(lambda x: "Shellcode/Reconnaissance" if x == "Shellcode" or x == "Reconnaissance" else x)
data_reduced.attack_cat = data_reduced.attack_cat.apply(lambda x: "Fuzzers/DoS" if x == "Fuzzers" or x == "DoS" else x)
data_reduced.attack_cat = data_reduced.attack_cat.apply(lambda x: "Analysis/Backdoor" if x == "Analysis" or x == "Backdoor" or x == "Backdoors" else x)

data_summary_classes = pd.DataFrame(data_reduced.attack_cat.value_counts())
data_summary_classes.columns.values[0] = "Occurrencies"

# attack_len = sum(data.Label)
data_summary_classes["Percentage"] = data_reduced.attack_cat.value_counts() / len(data_reduced)

data_summary_classes
# len(data_reduced)

## Adding non attack rows

In [ ]:
data = pd.concat([data_reduced,data[data.Label == 0].sample(100_000)])

## Plot the features correlation

In [ ]:
f,ax = plt.subplots(figsize=(40,32))

data_without_label = data.drop(columns=['Label'])

correlation_matrix = data_without_label.corr()

sns.heatmap(correlation_matrix, annot=True, linewidths=0.5, fmt= '.2f',ax=ax)
plt.show()

Drop the over-correlated features

In [ ]:
import numpy as np

abs_correlation_matrix = correlation_matrix.abs()

# Select upper triangle of correlation matrix
upper = abs_correlation_matrix.where(np.triu(np.ones(abs_correlation_matrix.shape), k=1).astype(np.bool_))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

# print(to_drop)

# Drop features 
data.drop(to_drop, axis=1, inplace=True)

print(data.info())
f,ax = plt.subplots(figsize=(40,32))

data_without_label = data.drop(columns=['Label'])

correlation_matrix = data_without_label.corr()

sns.heatmap(correlation_matrix, annot=True, linewidths=0.5, fmt= '.2f',ax=ax)
plt.show()

## Export preprocessed data to csv

In [ ]:
data.to_csv("./UNSW-NB15-preprocessed.csv")

data.info()